In [1]:
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
# Replace with your actual credentials
user = 'root'
password = 'root'
host = '172.20.0.2'
port = '5432'
database = 'magic_db'
table_name = 'prediction_data'


In [3]:
# Create connection
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}')

In [4]:
# Load data into DataFrame
df = pd.read_sql(f'SELECT * FROM {table_name}', engine)


In [5]:
df

,sensor_timestamp,modis_timestamp,sensor_value,fire_lat,fire_long,bright_ti4,confidence,fire_radiative_power,daynight,fire_detected
0,2025-04-08 00:00:00,NaT,102,NaN,NaN,NaN,None,NaN,None,0
1,2025-04-08 00:01:00,NaT,435,NaN,NaN,NaN,None,NaN,None,0
2,2025-04-08 00:02:00,NaT,860,NaN,NaN,NaN,None,NaN,None,0
3,2025-04-08 00:03:00,NaT,270,NaN,NaN,NaN,None,NaN,None,0
4,2025-04-08 00:04:00,NaT,106,NaN,NaN,NaN,None,NaN,None,0
...,...,...,...,...,...,...,...,...,...,...
356,2025-04-08 05:56:00,2025-04-08 05:55:00,337,60.71598,72.51321,325.73,n,5.05,D,0
357,2025-04-08 05:57:00,NaT,871,NaN,NaN,NaN,None,NaN,None,0
358,2025-04-08 05:58:00,NaT,640,NaN,NaN,NaN,None,NaN,None,0
359,2025-04-08 05:59:00,NaT,778,NaN,NaN,NaN,None,NaN,None,0


In [6]:
df.describe()

,sensor_timestamp,modis_timestamp,sensor_value,fire_lat,fire_long,bright_ti4,fire_radiative_power,fire_detected
count,361,137,361.000000,137.000000,137.000000,137.000000,137.000000,361.000000
mean,2025-04-08 03:00:00.000000256,2025-04-08 03:41:53.868613120,520.218837,14.811396,58.803403,325.590584,7.711095,0.005540
min,2025-04-08 00:00:00,2025-04-08 01:00:00,1.000000,-42.506610,-72.634570,296.410000,0.360000,0.000000
25%,2025-04-08 01:30:00,2025-04-08 02:43:00,270.000000,-13.198290,4.896310,305.390000,1.500000,0.000000
50%,2025-04-08 03:00:00,2025-04-08 03:51:00,510.000000,14.968940,72.513210,329.500000,3.230000,0.000000
75%,2025-04-08 04:30:00,2025-04-08 05:23:00,766.000000,45.256000,121.132450,338.700000,6.990000,0.000000
max,2025-04-08 06:00:00,2025-04-08 05:55:00,1023.000000,64.647530,174.405870,367.000000,82.550000,1.000000
std,NaN,NaN,289.186789,34.096629,69.396533,18.268798,13.864951,0.074329


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 361 entries, 0 to 360
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   sensor_timestamp      361 non-null    datetime64[ns]
 1   modis_timestamp       137 non-null    datetime64[ns]
 2   sensor_value          361 non-null    int64         
 3   fire_lat              137 non-null    float64       
 4   fire_long             137 non-null    float64       
 5   bright_ti4            137 non-null    float64       
 6   confidence            137 non-null    object        
 7   fire_radiative_power  137 non-null    float64       
 8   daynight              137 non-null    object        
 9   fire_detected         361 non-null    int64         
dtypes: datetime64[ns](2), float64(4), int64(2), object(2)
memory usage: 28.3+ KB


In [8]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Assuming df is your DataFrame

# Step 1: Handle Missing Data
# Fill missing values for numerical columns (fire_lat, fire_long, bright_ti4, fire_radiative_power)
df['fire_lat'].fillna(df['fire_lat'].median(), inplace=True)
df['fire_long'].fillna(df['fire_long'].median(), inplace=True)
df['bright_ti4'].fillna(df['bright_ti4'].median(), inplace=True)
df['fire_radiative_power'].fillna(df['fire_radiative_power'].median(), inplace=True)

# Fill missing values for categorical columns (confidence, daynight)
df['confidence'].fillna(df['confidence'].mode()[0], inplace=True)
df['daynight'].fillna(df['daynight'].mode()[0], inplace=True)

# Step 2: Feature Engineering
# Extract hour, day, and week from the 'sensor_timestamp' and 'modis_timestamp'
df['sensor_hour'] = df['sensor_timestamp'].dt.hour
df['sensor_dayofweek'] = df['sensor_timestamp'].dt.dayofweek
df['sensor_day'] = df['sensor_timestamp'].dt.day

# Calculate the difference between sensor_timestamp and modis_timestamp in minutes
df['timestamp_diff'] = (df['sensor_timestamp'] - df['modis_timestamp']).dt.total_seconds() / 60
df['timestamp_diff'].fillna(0, inplace=True)  # Fill NaT (Not a Time) with 0

# Step 3: Encode Categorical Data
# Encode the 'confidence' and 'daynight' columns
label_encoder = LabelEncoder()
df['confidence_encoded'] = label_encoder.fit_transform(df['confidence'])
df['daynight_encoded'] = label_encoder.fit_transform(df['daynight'])

# Step 4: Scaling/Normalization
scaler = StandardScaler()
df[['sensor_value', 'fire_lat', 'fire_long', 'fire_radiative_power']] = scaler.fit_transform(
    df[['sensor_value', 'fire_lat', 'fire_long', 'fire_radiative_power']])

# Step 5: Final DataFrame with necessary features
final_df = df.drop(columns=['sensor_timestamp', 'modis_timestamp', 'confidence', 'daynight'])
final_df.head()


/tmp/ipykernel_99673/4003657606.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['fire_lat'].fillna(df['fire_lat'].median(), inplace=True)
/tmp/ipykernel_99673/4003657606.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplac

,sensor_value,fire_lat,fire_long,bright_ti4,fire_radiative_power,fire_detected,sensor_hour,sensor_dayofweek,sensor_day,timestamp_diff,confidence_encoded,daynight_encoded
0,-1.448196,0.002857,0.120686,329.5,-0.193611,0,0,1,8,0.0,2,0
1,-0.295093,0.002857,0.120686,329.5,-0.193611,0,0,1,8,0.0,2,0
2,1.176585,0.002857,0.120686,329.5,-0.193611,0,0,1,8,0.0,2,0
3,-0.866451,0.002857,0.120686,329.5,-0.193611,0,0,1,8,0.0,2,0
4,-1.434345,0.002857,0.120686,329.5,-0.193611,0,0,1,8,0.0,2,0


In [17]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE

# Step 1: Split the data into features (X) and target (y)
X = final_df.drop(columns=['fire_detected'])
y = final_df['fire_detected']

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 2: Check class distribution before applying SMOTE
print("Class distribution before SMOTE:")
print(y_train.value_counts())

# Step 3: Apply SMOTE to oversample the minority class if there are enough samples
smote = SMOTE(random_state=42)

# If the minority class has only one sample, SMOTE won't work, so handle this
if y_train.value_counts().min() > 1:
    X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
    print("SMOTE applied successfully.")
else:
    print("SMOTE cannot be applied. Insufficient samples in the minority class.")
    X_train_smote, y_train_smote = X_train, y_train  # Use the original data if SMOTE can't be applied

# Train Random Forest with class weights
rf_classifier_weighted = RandomForestClassifier(random_state=42, class_weight='balanced')
rf_classifier_weighted.fit(X_train, y_train)

# # Evaluate the model
# y_pred_weighted = rf_classifier_weighted.predict(X_test)

# # Print the evaluation results
# accuracy_weighted = accuracy_score(y_test, y_pred_weighted)
# print(f"Accuracy with class weights: {accuracy_weighted:.4f}")
# print("Classification Report with class weights:")
# print(classification_report(y_test, y_pred_weighted))
# Get predicted probabilities instead of classes
y_pred_prob = rf_classifier_weighted.predict_proba(X_test)[:, 1]

# Adjust the threshold to 0.1 (or other values) to favor class 1 predictions
y_pred_adjusted = (y_pred_prob >= 0.1).astype(int)

# Evaluate the adjusted model
accuracy_adjusted = accuracy_score(y_test, y_pred_adjusted)
print(f"Accuracy with adjusted threshold: {accuracy_adjusted:.4f}")
print("Classification Report with adjusted threshold:")
print(classification_report(y_test, y_pred_adjusted))

Class distribution before SMOTE:
fire_detected
0    287
1      1
Name: count, dtype: int64
SMOTE cannot be applied. Insufficient samples in the minority class.
Accuracy with adjusted threshold: 1.0000
Classification Report with adjusted threshold:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        72
           1       1.00      1.00      1.00         1

    accuracy                           1.00        73
   macro avg       1.00      1.00      1.00        73
weighted avg       1.00      1.00      1.00        73

